In [1]:
from pytrends.request import TrendReq
import pandas as pd
import sqlite3
import numpy as np

In [2]:
#obtendo as tabelas de indicadores
conn =sqlite3.connect("indicadores.db")
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())
brasilDf=pd.read_sql_query("select * from brazil_df",conn)
conn.close()

[('imedreg_df',), ('cities_df',), ('states_df',), ('regions_df',), ('brazil_df',)]


In [3]:
#lendo o arquivo com as keywords
keywords=pd.read_csv('keywords.csv',header=None)
keywords

,0
0,['ansiedade']
1,['auxilio desemprego']
2,['azitromicina']
3,['brasil coronavirus mortes']
4,['brasil coronavirus']
...,...
119,['tosse']
120,['vacina china']
121,['vacina corona']
122,['álcool 70']


In [4]:
#gerando a lista de keywords
kw_list=[]
for i in range(len(keywords)):
    keywords[0].iloc[i]=keywords[0].iloc[i].replace('[','')
    keywords[0].iloc[i]=keywords[0].iloc[i].replace(']','')
    keywords[0].iloc[i]=keywords[0].iloc[i].replace('\'','')
    kw_list.append(keywords[0].iloc[i])

In [5]:
#como pode ser visto,a ultima semana totalmente em 2020 termina no dia 26 de dezembro e a primeira com dados
#começa dia 23 de fevereiro
print(brasilDf['week_last_day'].head())
print(brasilDf['week_last_day'].tail())

0    2020-02-29
1    2020-03-07
2    2020-03-14
3    2020-03-21
4    2020-03-28
Name: week_last_day, dtype: object
42    2020-12-19
43    2020-12-26
44    2021-01-02
45    2021-01-09
46    2021-01-16
Name: week_last_day, dtype: object


In [6]:
pytrends = TrendReq(hl='en-US', tz=360)

In [7]:
#dados das semanas 2020-02-23 até 2020-12-26 que são as semanas epidemiologicas 9 até a 52 relativos ao brasil
df=pd.DataFrame()
for i in range(len(kw_list)):
    kw = []
    kw.append(kw_list[i])
    pytrends.build_payload(kw, cat=0, timeframe='2020-02-23 2020-12-26', geo='BR', gprop='')
    aux=pytrends.interest_over_time()
    df[kw_list[i]]=aux[kw_list[i]]

KeyboardInterrupt: 

In [ ]:
df.to_csv('SeriesKeywords.csv')

In [ ]:
#dados das semanas 2020-02-23 até 2020-12-26 que são as semanas epidemiologicas 9 até a 52 relativos a MG
dfMg=pd.DataFrame()
for i in range(len(kw_list)):
    kw = []
    kw.append(kw_list[i])
    pytrends.build_payload(kw, cat=0, timeframe='2020-02-23 2020-12-26', geo='BR-MG', gprop='')
    aux=pytrends.interest_over_time()
    #caso a consulta não consiga resultado, criará uma coluna de NAN
    if not aux.empty:
        dfMg[kw_list[i]]=aux[kw_list[i]]
    else:
        dfMg[kw_list[i]]=np.nan

In [ ]:
dfMg.to_csv('SeriesKeywordsMg.csv')